# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document

Submitted by Chen Moasis


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

#NOT INCLUDED IN ORIGINAL LIST but are from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

#### Explanation about sklearn imports that were added:
#### Logistic Regression: 
A statistical method used for binary classification problems, where the goal is to predict whether an instance belongs to one of two possible classes. It models the probability of the positive class using a sigmoid function, and learns the optimal weights that minimize the binary cross-entropy loss.

#### MaxAbsScaler: 
A data preprocessing technique used to scale the features of a dataset to a range between -1 and 1, by dividing each value by the maximum absolute value of the corresponding feature. It's useful for sparse datasets or features with different scales, as it preserves the sparsity and does not shift the distribution. It's commonly used in conjunction with linear models, such as logistic regression or support vector machines, that assume standardized features.

#### StratifiedKFold:
A cross-validation technique that divides a dataset into K folds while preserving the original class distribution. It ensures a fair evaluation of models on imbalanced datasets. By training the model K times on different folds, it provides robust performance estimation.


#### MakeScorer: 
A function in scikit-learn that converts a performance metric function into a scorer object that can be used for model evaluation. It allows you to customize the scoring metric and parameters, such as average method for multi-class problems. The scorer object can be used with cross-validation or grid search functions to assess and compare models based on the specified metric.

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [4]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [6]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

C:\Users\חנושית\Downloads


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [7]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [8]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [9]:
df_train['gender'].value_counts()

m    575
f    178
Name: gender, dtype: int64

In [10]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [11]:
def clean_text(text):
    # Remove numbers, spaces, and leading/trailing spaces
    text = re.sub(r'\d+', '', text)
    # Remove non-Hebrew letters
    text = re.sub(r'[^\u0590-\u05FF\s]', '', text)
    # Replace consecutive spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # Strip leading/trailing spaces
    text = text.strip()
    return text

In [12]:
def tokenize_hebrew_text(text):
    tokens = ht.tokenize(text)
    return [token for _, token, _, _ in tokens]

In [13]:
models_params = {
        'Perceptron': {
        'vectorizer': [TfidfVectorizer(ngram_range=(1,3),min_df=5, max_df=435,tokenizer=tokenize_hebrew_text)],
        'scaler': [StandardScaler(with_mean=False)],
        'classifier': [Perceptron(penalty='l1')]
        },
       'lr': {
        'vectorizer': [TfidfVectorizer(tokenizer=tokenize_hebrew_text, min_df=5,max_df=428)],
        'vectorizer__max_features': [2000],
        'vectorizer__ngram_range': [(1, 1)],
        'scaler': [StandardScaler(with_mean=False)],
        'classifier': [LogisticRegression(class_weight='balanced')],
        'classifier__penalty': ['l2']
        },
        'nb': {
        'vectorizer': [CountVectorizer(tokenizer=tokenize_hebrew_text,min_df=5, max_df=400)],
        'vectorizer__max_features':[1000],
        'vectorizer__ngram_range': [(1, 2)],
        'scaler': [StandardScaler(with_mean=False)],
        'classifier': [MultinomialNB()],
        'classifier__alpha': [0.01],
        },
        'MLPClassifier': {
        'vectorizer': [TfidfVectorizer(tokenizer=tokenize_hebrew_text)],
        'vectorizer__max_features':[800],
        'vectorizer__ngram_range': [(1, 2)],
        'scaler': [MaxAbsScaler()],
        'classifier': [MLPClassifier()],
        'classifier__activation': ['logistic', 'tanh', 'relu'],
        'classifier__solver': ['lbfgs'],
        'classifier__alpha': [0.0001]
        },
        'LinearSVC': {
        'vectorizer': [CountVectorizer(tokenizer=tokenize_hebrew_text,min_df=5, max_df=435), TfidfVectorizer(tokenizer=tokenize_hebrew_text)],
        'vectorizer__ngram_range': [(1, 1), (1, 2),(1,3)],
        'scaler': [StandardScaler(with_mean=False), MinMaxScaler(), MaxAbsScaler()],
        'classifier': [LinearSVC()],
        'classifier__C': [0.01, 1, 10],
        'classifier__penalty': ['l2'],
        'classifier__tol': [0.001],
        },
     'knn': {
        'vectorizer': [CountVectorizer(tokenizer=tokenize_hebrew_text,min_df=5, max_df=435), TfidfVectorizer(tokenizer=tokenize_hebrew_text)],
        'vectorizer__max_features':[None, 1000],
        'vectorizer__ngram_range': [(1, 1), (1, 2),(1,3)],
        'scaler': [StandardScaler(with_mean=False), MinMaxScaler(), MaxAbsScaler()],
        'classifier': [KNeighborsClassifier()],
        'classifier__n_neighbors': [5, 50],
        'classifier__p': [1, 2]
        },
    'cart': {
        'vectorizer': [CountVectorizer(tokenizer=tokenize_hebrew_text,min_df=5, max_df=500)],
        'vectorizer__max_features':[None],
        'vectorizer__ngram_range': [(1,3)],
        'scaler': [StandardScaler(with_mean=False)],
        'classifier': [DecisionTreeClassifier()],
    },
}

In [14]:
# Split the data
X = df_train["story"]
y = df_train["gender"]
X = X.apply(clean_text)
y = y.apply(lambda x: 1 if x == 'f' else 0)  # female is 1

# Define the pipeline
pipeline = Pipeline([
    ('vectorizer', None),
    ('scaler', None),
    ('classifier', None)
])

# Define the cross-validation strategy
cv = StratifiedKFold()

# Define the f1 macro scorer
f1_macro_scorer = make_scorer(f1_score, average='macro')

# Perform grid search over the models_params dictionary
results = []
for model_name, model_params in models_params.items():
    print(f"\n-------------Grid search for {model_name}:-------------\n")
    clf = GridSearchCV(pipeline, model_params, cv=cv, scoring=f1_macro_scorer, verbose=1,n_jobs=10)
    clf.fit(X, y)

    # Get the best hyperparameters
    best_params = clf.best_params_

    # Perform cross-validation with the specified scoring method
    cv_scores = cross_val_score(clf.best_estimator_, X, y, scoring=f1_macro_scorer, cv=10)

    # Calculate the mean of cv_scores
    cv_mean_score = np.mean(cv_scores)

    # Store the results
    result = {
        "model_name": model_name,
        "best_params": best_params,
        "cv_mean_score": cv_mean_score
    }
    results.append(result)

    # Print the results for the current clf
    print(f"Best hyperparameters: {best_params}")
    print(f"Cross-validated F1 macro mean score: {cv_mean_score:.3f}") 
    print("\n--------------------------------------------------------\n")

# Print the final results
for result in results:
    print(f"Model: {result['model_name']}")
    print(f"Best hyperparameters: {result['best_params']}")
    print(f"Cross-validated F1 macro mean score: {result['cv_mean_score']:.3f}") 
    print("\n--------------------------------------------------------\n")


-------------Grid search for Perceptron:-------------

Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('vectorizer', None), ('scaler', None),
                                       ('classifier', None)]),
             n_jobs=10,
             param_grid={'classifier': [Perceptron(penalty='l1')],
                         'scaler': [StandardScaler(with_mean=False)],
                         'vectorizer': [TfidfVectorizer(max_df=435, min_df=5,
                                                        ngram_range=(1, 3),
                                                        tokenizer=<function tokenize_hebrew_text at 0x0000020451D87310>)]},
             scoring=make_scorer(f1_score, average=macro), verbose=1)

Best hyperparameters: {'classifier': Perceptron(penalty='l1'), 'scaler': StandardScaler(with_mean=False), 'vectorizer': TfidfVectorizer(max_df=435, min_df=5, ngram_range=(1, 3),
                tokenizer=<function tokenize_hebrew_text at 0x0000020451D87310>)}
Cross-validated F1 macro mean score: 0.718

--------------------------------------------------------


-------------Grid search for lr:-------------

Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('vectorizer', None), ('scaler', None),
                                       ('classifier', None)]),
             n_jobs=10,
             param_grid={'classifier': [LogisticRegression(class_weight='balanced')],
                         'classifier__penalty': ['l2'],
                         'scaler': [StandardScaler(with_mean=False)],
                         'vectorizer': [TfidfVectorizer(max_df=428,
                                                        max_features=2000,
                                                        min_df=5,
                                                        tokenizer=<function tokenize_hebrew_text at 0x0000020451D87310>)],
                         'vectorizer__max_features': [2000],
                         'vectorizer__ngram_range': [(1, 1)]},
             scoring=make_scorer(f1_score, average=macro), verbose=1)

Best hyperparameters: {'classifier': LogisticRegression(class_weight='balanced'), 'classifier__penalty': 'l2', 'scaler': StandardScaler(with_mean=False), 'vectorizer': TfidfVectorizer(max_df=428, max_features=2000, min_df=5,
                tokenizer=<function tokenize_hebrew_text at 0x0000020451D87310>), 'vectorizer__max_features': 2000, 'vectorizer__ngram_range': (1, 1)}
Cross-validated F1 macro mean score: 0.699

--------------------------------------------------------


-------------Grid search for nb:-------------

Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Best hyperparameters: {'classifier': LinearSVC(C=10, tol=0.001), 'classifier__C': 10, 'classifier__penalty': 'l2', 'classifier__tol': 0.001, 'scaler': StandardScaler(with_mean=False), 'vectorizer': CountVectorizer(max_df=435, min_df=5,
                tokenizer=<function tokenize_hebrew_text at 0x0000020451D87310>), 'vectorizer__ngram_range': (1, 1)}
Cross-validated F1 macro mean score: 0.653

--------------------------------------------------------


-------------Grid search for knn:-------------

Fitting 5 folds for each of 144 candidates, totalling 720 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('vectorizer', None), ('scaler', None),
                                       ('classifier', None)]),
             n_jobs=10,
             param_grid={'classifier': [KNeighborsClassifier()],
                         'classifier__n_neighbors': [5, 50],
                         'classifier__p': [1, 2],
                         'scaler': [StandardScaler(with_mean=False),
                                    MinMaxScaler(), MaxAbsScaler()],
                         'v...ntVectorizer(max_df=435, min_df=5,
                                                        tokenizer=<function tokenize_hebrew_text at 0x0000020451D87310>),
                                        TfidfVectorizer(max_features=1000,
                                                        ngram_range=(1, 3),
                                                        tokenizer=<function tokenize_hebrew_text at 0x0

Best hyperparameters: {'classifier': KNeighborsClassifier(), 'classifier__n_neighbors': 5, 'classifier__p': 2, 'scaler': StandardScaler(with_mean=False), 'vectorizer': TfidfVectorizer(max_features=1000, ngram_range=(1, 3),
                tokenizer=<function tokenize_hebrew_text at 0x0000020451D87310>), 'vectorizer__max_features': 1000, 'vectorizer__ngram_range': (1, 3)}
Cross-validated F1 macro mean score: 0.542

--------------------------------------------------------


-------------Grid search for cart:-------------

Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('vectorizer', None), ('scaler', None),
                                       ('classifier', None)]),
             n_jobs=10,
             param_grid={'classifier': [DecisionTreeClassifier()],
                         'scaler': [StandardScaler(with_mean=False)],
                         'vectorizer': [CountVectorizer(max_df=500, min_df=5,
                                                        ngram_range=(1, 3),
                                                        tokenizer=<function tokenize_hebrew_text at 0x0000020451D87310>)],
                         'vectorizer__max_features': [None],
                         'vectorizer__ngram_range': [(1, 3)]},
             scoring=make_scorer(f1_score, average=macro), verbose=1)

Best hyperparameters: {'classifier': DecisionTreeClassifier(), 'scaler': StandardScaler(with_mean=False), 'vectorizer': CountVectorizer(max_df=500, min_df=5, ngram_range=(1, 3),
                tokenizer=<function tokenize_hebrew_text at 0x0000020451D87310>), 'vectorizer__max_features': None, 'vectorizer__ngram_range': (1, 3)}
Cross-validated F1 macro mean score: 0.608

--------------------------------------------------------

Model: Perceptron
Best hyperparameters: {'classifier': Perceptron(penalty='l1'), 'scaler': StandardScaler(with_mean=False), 'vectorizer': TfidfVectorizer(max_df=435, min_df=5, ngram_range=(1, 3),
                tokenizer=<function tokenize_hebrew_text at 0x0000020451D87310>)}
Cross-validated F1 macro mean score: 0.718

--------------------------------------------------------

Model: lr
Best hyperparameters: {'classifier': LogisticRegression(class_weight='balanced'), 'classifier__penalty': 'l2', 'scaler': StandardScaler(with_mean=False), 'vectorizer': TfidfVector

In [15]:
# Update the pipeline with the best estimator
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(max_df=435, min_df=5, ngram_range=(1, 3), tokenizer= tokenize_hebrew_text)),
    ('scaler', StandardScaler(with_mean=False)),
    ('classifier', Perceptron(penalty='l1'))
])

# # Fit the pipeline to the training data with the best estimator
pipeline.fit(df_train['story'].apply(clean_text), y) # y is binary (1 for female)

# Make predictions on the test data
y_pred_test = pipeline.predict(df_test['story'].apply(clean_text))

# Sample first and last 5 rows of df_test and predict on them
df_pred_test_sample = pd.concat([df_test.head(), df_test.tail()])
df_pred_test_sample['predictions'] = np.concatenate([y_pred_test[:5], y_pred_test[-5:]])
df_pred_test_sample['predictions'] = pd.Series(y_pred_test).map({0: 'm', 1: 'f'})

df_pred_test_sample

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(max_df=435, min_df=5, ngram_range=(1, 3),
                                 tokenizer=<function tokenize_hebrew_text at 0x0000020451D87310>)),
                ('scaler', StandardScaler(with_mean=False)),
                ('classifier', Perceptron(penalty='l1'))])

,test_example_id,story,predictions
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...,m
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""...",m
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...,m
3,3,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני...",m
4,4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...,m
318,318,"בשנה האחרונה הרגשתי די תקוע בעבודה, השגרה הפכה...",m
319,319,אני ואילן חברים טובים מזה 20 שנה תמיד חלמנו לפ...,m
320,320,מידי יום שישי אני נוהג לנסוע בתחבורה ציבורית ס...,m
321,321,"לפני מספר חודשים, בשיא התחלואה של הגל השני, עמ...",m
322,322,היום בו דיווחתי על גניבה של האוטו שלי. בוקר אח...,m


In [16]:
best_result = max(results, key=lambda r: r['cv_mean_score'])
best_model_name = best_result['model_name']
best_params = best_result['best_params']
f1_score = best_result['cv_mean_score']

print(f"Best Model: {best_model_name}")
print(f"F1 mean Score: {f1_score:.4f}")
print("Best Parameters:")
for param, value in best_params.items():
    print(f"{param}: {value}")
results_table = pd.DataFrame(results)[['model_name', 'cv_mean_score']]
results_table = results_table.sort_values(by='cv_mean_score', ascending=False).reset_index(drop=True)
results_table

Best Model: Perceptron
F1 mean Score: 0.7176
Best Parameters:
classifier: Perceptron(penalty='l1')
scaler: StandardScaler(with_mean=False)
vectorizer: TfidfVectorizer(max_df=435, min_df=5, ngram_range=(1, 3),
                tokenizer=<function tokenize_hebrew_text at 0x0000020451D87310>)


,model_name,cv_mean_score
0,Perceptron,0.718
1,lr,0.699
2,LinearSVC,0.653
3,nb,0.649
4,MLPClassifier,0.647
5,cart,0.608
6,knn,0.542


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [17]:
df_pred_test_sample.to_csv('classification_results.csv',index=False)